In [ ]:
import os
import pickle
import time

from lightautoml.dataset.np_pd_dataset_cupy import *
from lightautoml.dataset.roles import *
from lightautoml.dataset.utils import roles_parser

from lightautoml.tasks import Task

In [ ]:
from lightautoml.transformers import numeric_gpu, categorical_gpu, datetime_gpu

# Initial data load

In [ ]:
data = pd.read_csv('./example_data/test_data_files/sampled_app_train.csv',
                       usecols=['TARGET', 'NAME_CONTRACT_TYPE', 'AMT_CREDIT',
                                'NAME_TYPE_SUITE', 'AMT_GOODS_PRICE',
                                'DAYS_BIRTH', 'DAYS_EMPLOYED'])

# Fix dates and convert to date type

data['BIRTH_DATE'] = np.datetime64('2018-01-01') + data['DAYS_BIRTH'].astype(np.dtype('timedelta64[D]'))
data['EMP_DATE'] = np.datetime64('2018-01-01') + np.clip(data['DAYS_EMPLOYED'], None, 0).astype(np.dtype('timedelta64[D]'))
data.drop(['DAYS_BIRTH', 'DAYS_EMPLOYED'], axis=1, inplace=True)

# Create folds
data['__fold__'] = np.random.randint(0, 5, len(data))

# Print data head
print(data.head())

# # Set roles for columns

check_roles = {
    TargetRole(): 'TARGET',
    CategoryRole(dtype=str): ['NAME_CONTRACT_TYPE', 'NAME_TYPE_SUITE'],
    NumericRole(np.float32): ['AMT_CREDIT', 'AMT_GOODS_PRICE'],
    DatetimeRole(seasonality=['y', 'm', 'wd']): ['BIRTH_DATE', 'EMP_DATE'],
    FoldsRole(): '__fold__'
}

# create Task
task = Task('binary')

# # Creating PandasDataSet
pd_dataset = PandasDataset(data, roles_parser(check_roles), task=task)

cudf_dataset = pd_dataset.to_cudf()

In [ ]:
np.isnan(data.NAME_TYPE_SUITE.iloc[180])

In [ ]:
# check for cudf to cupy values conversion
print(cudf_dataset.data['AMT_GOODS_PRICE'].values)

# check for nan correct representation
print(cudf_dataset.data['AMT_GOODS_PRICE'].values.sum())

# Numeric dataset creation & numerical transformers test

In [ ]:
check_roles = {
    NumericRole(np.float32): ['AMT_CREDIT', 'AMT_GOODS_PRICE'],
}

numeric_dataset = CudfDataset(data=cudf_dataset.data[['AMT_CREDIT',\
                                                      'AMT_GOODS_PRICE']].copy(),
                        roles=roles_parser(check_roles),
                        task=task)

In [ ]:
numeric_dataset.folds

## FillNA transformer

In [ ]:
filler = numeric_gpu.FillnaMedian()

In [ ]:
filled_dataset = filler.fit_transform(numeric_dataset)
print(filled_dataset)

## Standard scaler transformer

In [ ]:
scaler = numeric_gpu.StandardScaler()

In [ ]:
scaler.fit_transform(filled_dataset)

## NANRate

In [ ]:
nan_rate = numeric_gpu.NaNFlags()

In [ ]:
nan_rate.fit_transform(numeric_dataset)

## FillInf transformer

In [ ]:
fill_inf = numeric_gpu.FillInf()

In [ ]:
test_dataset = numeric_dataset.to_numpy().to_cupy()

In [ ]:
test_dataset.data[-1,0] = cp.inf

In [ ]:
fill_inf.fit_transform(test_dataset)

In [ ]:
test_dataset

## QuantileBinning transformer

In [ ]:
qb = numeric_gpu.QuantileBinning()

In [ ]:
qb.fit_transform(filled_dataset)

In [ ]:
filled_dataset

# Categorical dataset creation & categorical transformers test

In [ ]:
check_roles = {
    CategoryRole(dtype=str): ['NAME_CONTRACT_TYPE', 'NAME_TYPE_SUITE'],
}

In [ ]:
categorical_dataset = CudfDataset(data=cudf_dataset.data[['NAME_CONTRACT_TYPE',\
                                                      'NAME_TYPE_SUITE']].copy(),
                        roles=roles_parser(check_roles),
                        task=task)

In [ ]:
categorical_dataset.data.NAME_TYPE_SUITE.isna().sum()

## Label Encoder transformer

In [ ]:
le = categorical_gpu.LabelEncoder()

In [ ]:
type(categorical_dataset) == CudfDataset

In [ ]:
a = le.fit_transform(categorical_dataset).data

In [ ]:
encoded_dataset = le.transform(categorical_dataset)

In [ ]:
from lightautoml.transformers.categorical import LabelEncoder as LE

In [ ]:
label = LE()
b = label.fit_transform(encoded_dataset.to_cudf().to_pandas()).data

In [ ]:
(b == cp.asnumpy(a)).all()

## OneHot Encoder transformer

In [ ]:
ohe = categorical_gpu.OHEEncoder()

In [ ]:
ohe.fit_transform(encoded_dataset)

In [ ]:
from cuml.preprocessing import OneHotEncoder

In [ ]:
ohe_ = OneHotEncoder(categories='auto',
                                 dtype=cp.float32, sparse=False,
                                 handle_unknown='ignore')

In [ ]:
a = ohe_.fit_transform(encoded_dataset.data)

In [ ]:
b = ohe.fit_transform(encoded_dataset)

In [ ]:
(a == b.data).all()

In [ ]:
b.data

## FreqEncoder transformer

In [ ]:
freq_enc = categorical_gpu.FreqEncoder()

In [ ]:
freq_enc.fit_transform(encoded_dataset)

In [ ]:
from lightautoml.transformers.categorical import FreqEncoder as fe

In [ ]:
fe_ = fe()

In [ ]:
fe_.fit_transform(encoded_dataset.to_cudf().to_pandas())

In [ ]:
(freq_enc.fit_transform(encoded_dataset).to_numpy().data ==\
fe_.fit_transform(encoded_dataset.to_cudf().to_pandas()).data).all()

## TargetEncoder transformer

In [ ]:
encoded_dataset.folds = cudf_dataset.folds
encoded_dataset.target = cudf_dataset.target

In [ ]:
target_enc = categorical_gpu.TargetEncoder()

In [ ]:
a = target_enc.fit_transform(encoded_dataset)

In [ ]:
from lightautoml.transformers.categorical import TargetEncoder as te

In [ ]:
target_cpu = te()

encoded_cpu = encoded_dataset.to_cudf().to_pandas().to_numpy()
encoded_cpu.folds = cp.asnumpy(encoded_dataset.folds)
encoded_cpu.target = cp.asnumpy(encoded_dataset.target)
b = target_cpu.fit_transform(encoded_cpu)

In [ ]:
np.allclose(cp.asnumpy(a.data), b.data)

## Multiclass TargetEncoder transformer

In [ ]:
task = Task('multiclass')
check_roles = {
    CategoryRole(dtype=int): ['le__NAME_CONTRACT_TYPE', 'le__NAME_TYPE_SUITE'],
}

encoded_multiclass = CupyDataset(data=encoded_dataset.data.copy(),
                        features=['le__NAME_CONTRACT_TYPE', 'le__NAME_TYPE_SUITE'],
                        roles=roles_parser(check_roles),
                        task=task)

encoded_multiclass.target = cudf.Series(cp.random.randint(0,5,10000))

In [ ]:
le = categorical_gpu.LabelEncoder()
multi_te = categorical_gpu.MultiClassTargetEncoder()

In [ ]:
encoded_le = le.fit_transform(encoded_multiclass)
encoded_le.target = encoded_multiclass.target
encoded_le.folds = encoded_dataset.folds
a = multi_te.fit_transform(encoded_le).data

In [ ]:
encoded_cpu = encoded_le.to_cudf().to_pandas().to_numpy()
encoded_cpu.target = encoded_multiclass.target.to_pandas().values
encoded_cpu.folds = encoded_dataset.folds.to_pandas().values

In [ ]:
from lightautoml.transformers.categorical import MultiClassTargetEncoder as MCTE

In [ ]:
mlc = MCTE()

In [ ]:
b = mlc.fit_transform(encoded_cpu).data

In [ ]:
np.allclose(a, cp.asarray(b))

In [ ]:
(np.abs(cp.asnumpy(a)-b).max()/np.abs(b)).max()

## CatIntersections transformer

In [ ]:
cat_intersect_transformer = categorical_gpu.CatIntersectstions()
a = cat_intersect_transformer.fit_transform(encoded_dataset.to_cudf()).data

In [ ]:
from lightautoml.transformers.categorical import CatIntersectstions as CI
cat_int = CI()
b = cat_int.fit_transform(encoded_dataset.to_cudf().to_pandas()).data

In [ ]:
print((cp.asnumpy(a)==b).mean())

## Ordinal Encoder transformer

In [ ]:
ordinal_encoder = categorical_gpu.OrdinalEncoder()

In [ ]:
a = ordinal_encoder.fit_transform(categorical_dataset).data

In [ ]:
from lightautoml.transformers.categorical import OrdinalEncoder as OE

In [ ]:
ordinal = OE()
b = ordinal.fit_transform(categorical_dataset.to_cudf().to_pandas()).data

In [ ]:
(cp.asnumpy(a) == b).all()

# Datetime transformers

In [ ]:
check_roles = {
    DatetimeRole(seasonality=['y', 'm', 'wd']): ['BIRTH_DATE', 'EMP_DATE'],
}

datetime_dataset = CudfDataset(data=cudf_dataset.data[['BIRTH_DATE',\
                                                      'EMP_DATE']].copy(),
                        roles=roles_parser(check_roles),
                        task=task)

In [ ]:
datetime_dataset.data

## TimeToNum transformer

In [ ]:
time_to_num = datetime_gpu.TimeToNum()

In [ ]:
time_to_num.fit_transform(datetime_dataset)

## BaseDiff transformer

In [ ]:
basediff = datetime_gpu.BaseDiff(base_names=['EMP_DATE'], diff_names=['BIRTH_DATE'])

In [ ]:
basediff.fit_transform(datetime_dataset)

## DateSeasons transformer

In [ ]:
date_seasons = datetime_gpu.DateSeasons()

date_dataset = date_seasons.fit_transform(datetime_dataset)

In [ ]:
type(numeric_dataset)

In [ ]:
date_dataset.to_cudf()

# ML Algos test

In [ ]:
from lightautoml.ml_algo import linear_gpu, linear_sklearn

In [ ]:
check_roles = {
    TargetRole(): 'TARGET',
}

# create Task
task = Task('binary')

# # Creating PandasDataSet
target_dataset = CudfDataset(data[['TARGET']], roles_parser(check_roles), task=task)

In [ ]:
filled_dataset = filled_dataset.to_cudf()
filled_dataset = filled_dataset.concat([filled_dataset, date_dataset.to_cudf(), target_dataset])
full_dataset = filled_dataset

In [ ]:
check_roles = {
    TargetRole(cp.float32): 'TARGET',
    NumericRole(cp.float32): ['fillnamed__AMT_CREDIT', 'fillnamed__AMT_GOODS_PRICE',
       'season_y__BIRTH_DATE', 'season_m__BIRTH_DATE', 'season_wd__BIRTH_DATE',
       'season_y__EMP_DATE', 'season_m__EMP_DATE', 'season_wd__EMP_DATE'],
}

In [ ]:
task = Task('binary', device='gpu')

In [ ]:
full_dataset = CupyDataset(filled_dataset.data.values,
                            features=filled_dataset.data.columns.to_list(),
                              roles=roles_parser(check_roles),
                              task=task,
                              **{'target': filled_dataset.data['TARGET']})

In [ ]:
from lightautoml.validation.utils import create_validation_iterator

In [ ]:
train_valid = create_validation_iterator(full_dataset[:9000], full_dataset[9000:], n_folds=10)

In [ ]:
linear_one = linear_gpu.LinearLBFGS()

In [ ]:
preds = linear_one.fit_predict(train_valid)

In [ ]:
y_true = full_dataset[9000:].target
y_pred = preds

In [ ]:
linear_cuml = linear_gpu.LinearL1CD()

In [ ]:
_=linear_cuml.fit_predict(train_valid)

In [ ]:
from cupyx.scipy.sparse import csr_matrix, hstack, coo_matrix

In [ ]:
a = csr_matrix(cp.eye(5))

In [ ]:
type(csr_matrix(a).toarray())

In [ ]:
a.shape

In [ ]:
numeric_dataset.to_numpy().to_sparse_gpu().data

In [ ]:
matrix = coo_matrix(a, dtype=cp.float32)

In [ ]:
matrix.data

In [ ]:
import torch

In [ ]:
b = torch.as_tensor(matrix.data, device='cuda')

In [ ]:
matrix.data[0] = 0

In [ ]:
b

In [ ]:
matrix.data

In [ ]:
a = [np.mean, np.sqrt]

In [ ]:
for f in a:
    print(f(np.array([2])))

In [ ]:
categorical_dataset.data.sample(100000, replace=True).reset_index().drop(['index'],axis=1)

In [ ]:
cudf_dataset.data.head()